In [ ]:
%pip install xhtml2pdf

In [18]:
from IPython.display import display, HTML
from xhtml2pdf import pisa 
from dotenv import load_dotenv
from sqlalchemy import create_engine
from datetime import datetime
from os import getenv
import pandas as pd

In [15]:
load_dotenv()

True

In [16]:
db_host = getenv('DB_HOST')
db_port = getenv('DB_PORT')
db_user = getenv('DB_USER')
db_password = getenv('DB_PASSWORD')
db_name = getenv('DB_NAME')
group_user = getenv('GROUP_USER')
group_user_pass = getenv('GROUP_USER_PASS')

In [17]:
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [20]:
con = engine.connect()

### Quering database

#### Number of rides completed in the past day

In [22]:
query = """
WITH todays_rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides;
    )
SELECT COUNT(*) AS number_of_rides 
FROM todays_rides 
WHERE start_date = '2022-10-07'
"""
number_of_rides = pd.read_sql_query(query, con)
number_of_rides

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near ";"
LINE 4:     FROM yusra_stories_production.rides;
                                               ^

[SQL: 
WITH todays_rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides;
    )
SELECT COUNT(*) AS number_of_rides 
FROM todays_rides 
WHERE start_date = '2022-10-07'
]
(Background on this error at: https://sqlalche.me/e/14/f405)

#### Gender split of riders of the past day

In [ ]:
query = """
    
"""
riders_gender_split = s.read_query()

#### Ages of the riders of the past day

#### Average power and heart rate of riders of past day

### Create the HTML Template

In [32]:
def report_block_template( graph_relative_path, caption=''):

    graph_block = (''
            '<a href="{graph_relative_path}" target="_blank">' # Open the interactive graph when you click on the image
                '<img style="height: 400px;" img src="{graph_relative_path}">'
            '</a>')

    report_block = ('' +
        graph_block +
        '{caption}' + 
        '<br>'      + 
        '</a>' +
        '<br>' +
        '<hr>')                       
    report_layout = (
       '<h2>Deloton Exercise Bikes Daily Report</h2>'
       + '<hr>'
       + report_block
    )
    return report_layout.format(graph_relative_path=graph_relative_path, caption=caption)


graph_relative_path = 'age_distr.png'

report = report_block_template(graph_relative_path, caption='Dis a graph')


In [33]:
display(HTML(report))

### Convert the HTML to PDF

In [28]:

def convert_html_to_pdf(source_html, output_filename):

    result_file = open(output_filename, "w+b")

    pisa_status = pisa.CreatePDF(
            source_html,           
            dest=result_file)           

    result_file.close()           

    return pisa_status.err

In [29]:
convert_html_to_pdf(report, 'report.pdf')

0

In [30]:
! open report.pdf